In [1]:
from pyalex import (
    Works, Authors, Sources,
    Institutions, Concepts, Publishers, Funders
)
import pyalex
import pandas as pd
import numpy as np
pyalex.config.email = "david@rs21.io"

from flair.embeddings import DocumentPoolEmbeddings
from flair.data import Sentence
from flair.embeddings import SentenceTransformerDocumentEmbeddings

EMBEDDING_MODEL_1 = "all-mpnet-base-v2" 

# this one is also good: all-MiniLM-L6-v2
EMBEDDING_MODEL_2 = "all-MiniLM-L6-v2"
SENT_EMBEDDINGS_1 = SentenceTransformerDocumentEmbeddings(EMBEDDING_MODEL_1)
SENT_EMBEDDINGS_2 = SentenceTransformerDocumentEmbeddings(EMBEDDING_MODEL_2)
DOC_EMBEDDINGS= DocumentPoolEmbeddings([SENT_EMBEDDINGS_2])

import torch
from tqdm import tqdm
import yake
import umap.umap_ as umap
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture as GMM
import altair as alt
import math
import plotly.express as px
import textwrap

In [2]:
search_term = 'missile'
robot_concepts = Concepts().search_filter(display_name=search_term).get()
len(robot_concepts)
for i in range(len(robot_concepts)):
    id_, display_name = robot_concepts[i]['id'], robot_concepts[i]['display_name']
    print(id_, display_name)

https://openalex.org/C2778857364 Missile
https://openalex.org/C522053795 Missile guidance
https://openalex.org/C122136912 Ballistic missile
https://openalex.org/C559253537 Missile defense
https://openalex.org/C28849524 Cruise missile
https://openalex.org/C202802212 Air-to-air missile


In [3]:
results, meta  = Concepts().get(return_meta=True)
print(meta)

{'count': 65073, 'db_response_time_ms': 30, 'page': 1, 'per_page': 25, 'groups_count': None}


In [4]:
search_term = 'jamming'
search_term = 'radar jamming and deception|electronic warfare|Network-centric warfare|Air-to-air missile'
search_term = ('radar jamming and deception|electronic warfare|Network-centric warfare|missile guidance' +
               '|Robot manipulator|Automatic target recognition' + 
              '|Pulsed power|High-energy X-rays' + 
              '|Ballistic missile|Air-to-air missile' + 
              '|terminal guidance')
jamming_concepts = Concepts().\
search_filter(display_name=search_term).get()

In [5]:
concepts = []
for i in range(len(jamming_concepts)):
    id_, display_name = jamming_concepts[i]['id'], jamming_concepts[i]['display_name']
    concepts.append((id_, display_name))
concepts

[('https://openalex.org/C2985527887', 'Robot manipulator'),
 ('https://openalex.org/C97039730', 'Pulsed power'),
 ('https://openalex.org/C522053795', 'Missile guidance'),
 ('https://openalex.org/C117623542', 'Automatic target recognition'),
 ('https://openalex.org/C122136912', 'Ballistic missile'),
 ('https://openalex.org/C176381164', 'Radar jamming and deception'),
 ('https://openalex.org/C183838350', 'High-energy X-rays'),
 ('https://openalex.org/C133082901', 'Electronic warfare'),
 ('https://openalex.org/C2777047555', 'Terminal guidance'),
 ('https://openalex.org/C2781187084', 'Network-centric warfare'),
 ('https://openalex.org/C202802212', 'Air-to-air missile')]

In [6]:
def process_works_list(worklist:list):
    """
    transforms the 
    works list into a dataframe.
    """
    abstracts_dict = {h["id"]:h["abstract"] for h in worklist}
    df = pd.DataFrame.from_records(worklist)
    try: 
        del df['abstract_inverted_index'] # though don't all have abstracts is the problem
        df['abstract'] = df['id'].map(abstracts_dict)
    except:
        pass
   # df['author_affils'] = df['authorships'].apply(get_authors_and_affils)
    return df

In [7]:
for i in range(len(jamming_concepts)):
    print(jamming_concepts[i]['id'], jamming_concepts[i]['works_count'])

https://openalex.org/C2985527887 10153
https://openalex.org/C97039730 14094
https://openalex.org/C522053795 6487
https://openalex.org/C117623542 3634
https://openalex.org/C122136912 6210
https://openalex.org/C176381164 2671
https://openalex.org/C183838350 1184
https://openalex.org/C133082901 3471
https://openalex.org/C2777047555 1355
https://openalex.org/C2781187084 1799
https://openalex.org/C202802212 1313


In [8]:
def get_hpm_frame():
    #hpm_pager = Works().filter(publication_year='>2020').search("high power microwave").\
#filter(authorships={"institutions":{"country_code":"CN"}}).paginate(per_page=200,
#                                                                    n_max=None)
    hpm_pager = Works().filter(publication_year='>2020').search("high power microwave").\
        paginate(per_page=200,  n_max=None)
    df = pd.DataFrame()
    for page in tqdm(hpm_pager):
        dfpage = process_works_list(page)
        df = pd.concat([df, dfpage], ignore_index=True)
        df.drop_duplicates(subset='id', keep='first',inplace=True)
    return df  

In [9]:
def get_sbl_frame():
    #sbl_pager = Works().filter(publication_year='>2020').search("space based laser").\
#filter(authorships={"institutions":{"country_code":"CN"}}).paginate(per_page=200,
#                                                                    n_max=None)
    sbl_pager = Works().filter(publication_year='>2020').search("space based laser").\
        paginate(per_page=200,
                                                                    n_max=None)
    df = pd.DataFrame()
    for page in tqdm(sbl_pager):
        dfpage = process_works_list(page)
        df = pd.concat([df, dfpage], ignore_index=True)
        df.drop_duplicates(subset='id', keep='first',inplace=True)
    return df    

In [10]:
def get_kkv_frame():
    #kkv_pager = Works().filter(publication_year='>2020').search("kinetic kill vehicle").\
#filter(authorships={"institutions":{"country_code":"CN"}}).paginate(per_page=200,
#                                                                    n_max=None)
    kkv_pager = Works().filter(publication_year='>2020').search("kinetic kill vehicle").\
        paginate(per_page=200,
                                                                    n_max=None)
    df = pd.DataFrame()
    for page in tqdm(kkv_pager):
        dfpage = process_works_list(page)
        df = pd.concat([df, dfpage], ignore_index=True)
        df.drop_duplicates(subset='id', keep='first',inplace=True)
    return df                                                               
    

In [11]:
def get_rka_frame():
   # rka_pager = Works().filter(publication_year='>2020').search("relativistic klystron amplifier").\
#filter(authorships={"institutions":{"country_code":"CN"}}).paginate(per_page=200,
#                                                                    n_max=None)
    rka_pager = Works().filter(publication_year='>2020').search("relativistic klystron amplifier").\
        paginate(per_page=200,
                                                                    n_max=None)
    df = pd.DataFrame()
    for page in tqdm(rka_pager):
        dfpage = process_works_list(page)
        df = pd.concat([df, dfpage], ignore_index=True)
        df.drop_duplicates(subset='id', keep='first',inplace=True)
    return df 

In [12]:
def get_concept_frame(concepts_list:list, i:int):
    """
    takes a list of Concepts() results and an index
    and forms the pagination object to retrive the 
    records
    """
    pager = Works().filter(publication_year='>2016',
    #concepts={"id":f"{concepts_list[i]['id']}"}).filter(authorships={"institutions":{"country_code":"CN"}}).\
    #paginate(per_page=200,n_max=None)
    concepts={"id":f"{concepts_list[i]['id']}"}).\
    paginate(per_page=200,n_max=None)
    df = pd.DataFrame()
    for page in tqdm(pager):
        dfpage = process_works_list(page)
        df = pd.concat([df, dfpage], ignore_index=True)
        df.drop_duplicates(subset='id', keep='first',inplace=True)
    return df

In [13]:
frames_list = []
for i in range(len(jamming_concepts)):
    df = get_concept_frame(jamming_concepts, i)
    frames_list.append(df)

14it [00:35,  2.53s/it]
16it [00:46,  2.88s/it]
8it [00:23,  2.98s/it]
8it [00:21,  2.69s/it]
7it [00:14,  2.12s/it]
6it [00:16,  2.71s/it]
2it [00:03,  1.97s/it]
6it [00:15,  2.50s/it]
3it [00:03,  1.03s/it]
3it [00:05,  1.74s/it]
2it [00:03,  1.67s/it]


In [14]:
frames_list[3]['created_date'].max()

'2024-05-14'

In [15]:
frames_list[2]['abstract'].value_counts(dropna=False)

None                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [16]:
len(frames_list)

11

In [17]:
dftop = pd.concat(frames_list,
                  ignore_index=True)
dftop.drop_duplicates(subset='id', keep='first', 
                      inplace=True)

dftop.set_index('id', inplace=True, drop=False)

dfall = dftop
print(dfall.shape)

dfall['content'] = dfall['title'] + ". " + dfall['abstract']

dfrecords = dfall[~dfall['content'].isna()].copy()

(10889, 49)


In [18]:
def get_keywords(text:str, top:int=7, stopwords=None):
    """
    takes a blob of text and 
    returns the top **top** 
    keywords as a list
    """
    kw_extractor = yake.KeywordExtractor(top=top, stopwords=stopwords)
    keywords = kw_extractor.extract_keywords(text)
    return [p[0] for p in keywords]

In [19]:
def get_top_concepts(concept_list:list,score:float=.6):
    """
    takes a list of concept dictionaries 
    returns the top **top** display_names;
    concepts whose score is >= score
    """
    return [c['display_name'] for c in concept_list if c['score'] >= score]

In [20]:
dfrecords['keywords'] = dfrecords['content'].apply(get_keywords)
dfrecords['top_concepts'] = dfrecords['concepts'].apply(get_top_concepts)

In [21]:
texts = dfrecords['content'].str.lower().values.tolist()

In [22]:
def get_content_embeddings(dfrecords:pd.DataFrame) -> pd.DataFrame:
    """
    passes the preprocessed mitigation strings
    data through the embedding model to produce the vector
    space representation of each pet mitigation.
    """
    sent = Sentence("The grass is green.")
    DOC_EMBEDDINGS.embed(sent)
    texts = dfrecords["content"].str.lower().values.tolist()
    all_descriptions = np.empty((len(texts), len(sent.embedding)))
    for i in tqdm(range(len(texts))):
        sent = Sentence(texts[i])
        DOC_EMBEDDINGS.embed(sent)
        all_descriptions[i, :] = sent.embedding.cpu().numpy()
        # gc.collect()
        torch.cuda.empty_cache()
    dfcontentvectors = pd.DataFrame.from_records(all_descriptions, index=dfrecords.index)
    return dfcontentvectors

In [23]:
dfcontentvectors = get_content_embeddings(dfrecords)

100%|█████████████████████████████████████████████████████████████████████| 9301/9301 [05:00<00:00, 31.00it/s]


In [24]:
#umap.UMAP?
N_COMPONENTS = 2 # can visualize this way
umap_reducer = umap.UMAP(n_components=N_COMPONENTS,
                       #  metric='euclidean')
                         random_state=1234,
                         metric='cosine')  # can experiment with this metric as well as the other 
# parameters
# to see what other literature is in the same information space, we need to keep this umap_reducer 
# object as well as the gmm model below.

# Apply UMAP to the vectorized strings
reduced_vectors = umap_reducer.fit_transform(dfcontentvectors.to_numpy())
dfreduced = pd.DataFrame.from_records(reduced_vectors, 
                index=dfcontentvectors.index)
dfreduced.columns = ['x','y']

/home/davidd/.local/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


# use hdbscan to cluster

In [25]:
import hdbscan

hdbscan_args = {'min_cluster_size': 15,
                            'metric': 'euclidean',
                            'cluster_selection_method': 'eom',
                            'cluster_selection_epsilon': 0.1
               }

cluster = hdbscan.HDBSCAN(**hdbscan_args).fit(dfreduced[['x','y']].to_numpy())

dfreduced['cluster'] = cluster.labels_
dfreduced['probability'] = cluster.probabilities_

dfpapers = dfrecords.merge(dfreduced, left_index=True,
                           right_index=True)

In [26]:
#help(dfpapers.explode)
del dfpapers['id']
dfstart = dfpapers.reset_index()
dfstart.head()

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,...,created_date,fulltext_origin,abstract,is_authors_truncated,content,top_concepts,x,y,cluster,probability
0,https://openalex.org/W1517236425,https://doi.org/10.1201/9781003062714,Neural Network Control Of Robot Manipulators A...,Neural Network Control Of Robot Manipulators A...,2020,2020-08-13,{'openalex': 'https://openalex.org/W1517236425...,en,"{'is_oa': False, 'landing_page_url': 'https://...",book,...,2016-06-24,NaN,"There has been great interest in ""universal co...",NaN,Neural Network Control Of Robot Manipulators A...,"[Robot manipulator, Artificial neural network]",16.062922,-3.320433,1,1.0
1,https://openalex.org/W2740675802,https://doi.org/10.1109/tcyb.2017.2711961,Adaptive Neural Network Control of a Robotic M...,Adaptive Neural Network Control of a Robotic M...,2017,2017-10-01,{'openalex': 'https://openalex.org/W2740675802...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,2017-08-08,ngrams,The control problem of an uncertain n -degrees...,NaN,Adaptive Neural Network Control of a Robotic M...,"[Control theory (sociology), Lyapunov function...",16.461128,-3.442048,1,1.0
2,https://openalex.org/W2418767125,https://doi.org/10.1109/tsmc.2016.2562506,Neural Network Control of a Flexible Robotic M...,Neural Network Control of a Flexible Robotic M...,2017,2017-08-01,{'openalex': 'https://openalex.org/W2418767125...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,2016-06-24,ngrams,Adaptive neural networks (NNs) are employed fo...,NaN,Neural Network Control of a Flexible Robotic M...,"[Control theory (sociology), Deflection (physi...",16.109781,-3.551340,1,1.0
3,https://openalex.org/W2901112449,https://doi.org/10.1109/tro.2018.2878318,Model-Based Reinforcement Learning for Closed-...,Model-Based Reinforcement Learning for Closed-...,2019,2019-02-01,{'openalex': 'https://openalex.org/W2901112449...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,2018-11-29,pdf,Dynamic control of soft robotic manipulators i...,NaN,Model-Based Reinforcement Learning for Closed-...,"[Control theory (sociology), Kinematics, Reinf...",15.096028,-5.377518,1,1.0
4,https://openalex.org/W2792852625,https://doi.org/10.1016/j.neucom.2018.01.002,Robot manipulator control using neural network...,Robot manipulator control using neural network...,2018,2018-04-01,{'openalex': 'https://openalex.org/W2792852625...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,2018-03-29,ngrams,Robot manipulators are playing increasingly si...,NaN,Robot manipulator control using neural network...,"[Artificial neural network, Computer science]",15.602239,-3.136606,1,1.0


In [27]:
dfstart['publication_year'].value_counts(dropna=False)

2022    1402
2023    1399
2021    1316
2019    1287
2018    1231
2020    1222
2017    1147
2024     297
Name: publication_year, dtype: int64

In [28]:
dfstart.shape

(9301, 55)

In [29]:
dfbig = dfstart.explode(column='authorships')
dfbig.shape, dfstart.shape

((37241, 55), (9301, 55))

In [30]:
dfbig.columns

Index(['id', 'doi', 'title', 'display_name', 'publication_year',
       'publication_date', 'ids', 'language', 'primary_location', 'type',
       'type_crossref', 'indexed_in', 'open_access', 'authorships',
       'countries_distinct_count', 'institutions_distinct_count',
       'corresponding_author_ids', 'corresponding_institution_ids', 'apc_list',
       'apc_paid', 'has_fulltext', 'cited_by_count',
       'cited_by_percentile_year', 'biblio', 'is_retracted', 'is_paratext',
       'primary_topic', 'topics', 'keywords', 'concepts', 'mesh',
       'locations_count', 'locations', 'best_oa_location',
       'sustainable_development_goals', 'grants', 'datasets', 'versions',
       'referenced_works_count', 'referenced_works', 'related_works',
       'ngrams_url', 'cited_by_api_url', 'counts_by_year', 'updated_date',
       'created_date', 'fulltext_origin', 'abstract', 'is_authors_truncated',
       'content', 'top_concepts', 'x', 'y', 'cluster', 'probability'],
      dtype='object')

In [31]:
dfbig.locations.iloc[68]

[{'is_oa': False,
  'landing_page_url': 'https://doi.org/10.1109/tsmc.2020.2999485',
  'pdf_url': None,
  'source': {'id': 'https://openalex.org/S4210209078',
   'display_name': 'IEEE transactions on systems, man, and cybernetics. Systems',
   'issn_l': '2168-2216',
   'issn': ['2168-2216', '2168-2232'],
   'is_oa': False,
   'is_in_doaj': False,
   'host_organization': 'https://openalex.org/P4310319808',
   'host_organization_name': 'Institute of Electrical and Electronics Engineers',
   'host_organization_lineage': ['https://openalex.org/P4310319808'],
   'host_organization_lineage_names': ['Institute of Electrical and Electronics Engineers'],
   'type': 'journal'},
  'license': None,
  'license_id': None,
  'version': None,
  'is_accepted': False,
  'is_published': False}]

In [32]:
def add_extra_to_authorships(row: pd.DataFrame):
    """
    row[authorships] is a dictionary;
    add in the id key to that dictionary
    whose value is row[id]
    """
    complete_dict = row["authorships"]
   # assert type(complete_dict) == dict
    #print(type(complete_dict))
    if type(complete_dict) == dict:
        complete_dict["id"] = row["id"]
        complete_dict["x"] = row["x"]
        complete_dict["y"] = row["y"]
        complete_dict["cluster"] = row["cluster"]
        complete_dict["cluster_score"] = row["probability"]
        complete_dict["title"] = row["title"]
        complete_dict["abstract"] = row["abstract"]
        complete_dict["doi"] = row["doi"]
        complete_dict["publication_date"] = row["publication_date"]
        complete_dict["publication_year"] = row["publication_year"]
        complete_dict["grants"] = row["grants"]
        complete_dict["locations"] = row["locations"]
        return complete_dict
    else:
        return row["authorships"]

In [33]:
dfbig['big_authorships'] = dfbig.apply(add_extra_to_authorships, axis=1)

In [34]:
dfbig.columns

Index(['id', 'doi', 'title', 'display_name', 'publication_year',
       'publication_date', 'ids', 'language', 'primary_location', 'type',
       'type_crossref', 'indexed_in', 'open_access', 'authorships',
       'countries_distinct_count', 'institutions_distinct_count',
       'corresponding_author_ids', 'corresponding_institution_ids', 'apc_list',
       'apc_paid', 'has_fulltext', 'cited_by_count',
       'cited_by_percentile_year', 'biblio', 'is_retracted', 'is_paratext',
       'primary_topic', 'topics', 'keywords', 'concepts', 'mesh',
       'locations_count', 'locations', 'best_oa_location',
       'sustainable_development_goals', 'grants', 'datasets', 'versions',
       'referenced_works_count', 'referenced_works', 'related_works',
       'ngrams_url', 'cited_by_api_url', 'counts_by_year', 'updated_date',
       'created_date', 'fulltext_origin', 'abstract', 'is_authors_truncated',
       'content', 'top_concepts', 'x', 'y', 'cluster', 'probability',
       'big_authorships'],

In [35]:
dfbig['authorships'].iloc[69] # raw_affiliation_strings    

{'author_position': 'middle',
 'author': {'id': 'https://openalex.org/A5029350515',
  'display_name': 'Wenkang Zhan',
  'orcid': None},
 'institutions': [{'id': 'https://openalex.org/I90610280',
   'display_name': 'South China University of Technology',
   'ror': 'https://ror.org/0530pts50',
   'country_code': 'CN',
   'type': 'education',
   'lineage': ['https://openalex.org/I90610280']}],
 'countries': ['CN'],
 'is_corresponding': False,
 'raw_author_name': 'Wenkang Zhan',
 'raw_affiliation_strings': ['School of Automation Science and Engineering South China University of Technology, Guangzhou 510640, China'],
 'id': 'https://openalex.org/W3037315148',
 'x': 16.364036560058594,
 'y': -4.629560470581055,
 'cluster': 1,
 'cluster_score': 0.987027185996989,
 'title': 'Boundary Control of a Rotating and Length-Varying Flexible Robotic Manipulator System',
 'abstract': 'This article copes with vibration suppression and angular position tracking problems of a robotic manipulator system com

In [36]:
#dfbig['authorships'].tolist()
bigvals = dfbig['authorships'].tolist()

In [37]:
dictvals = [c for c in bigvals if type(c) != float]

In [38]:
dictvals[0]['author'].keys()

dict_keys(['id', 'display_name', 'orcid'])

raw_affiliation_string -> raw_affiliation_strings

In [39]:
dftriple = pd.json_normalize(dictvals,
                  record_path=['institutions'],
                  meta=['id','raw_affiliation_strings','author_position', 'doi',
                        'title','abstract','publication_date', 'publication_year',
                        'grants','locations',
                        'is_corrresponding','x','y','cluster','cluster_score',
                       ['author','id'], ['author', 'display_name'],
                       ['author','orcid']],
                  errors='ignore',
                  sep='_',
                  meta_prefix='paper_',
                #  record_prefix='author_'
                 )

In [40]:
dftopics = dfcontentvectors.copy()
dftopics['cluster'] = dfpapers['cluster']
dfmeantopics = dftopics.groupby('cluster').mean().copy()
reduced_topics = umap_reducer.transform(dfmeantopics.to_numpy())
df_reduced_topics = pd.DataFrame.from_records(reduced_topics, 
                index=dfmeantopics.index)
df_reduced_topics.columns = ['x','y']
df_reduced_topics['topic'] = df_reduced_topics.index
df_reduced_topics.head()

def get_cluster_concepts(topic_num:int, n:int=20):
    """
    takes an integer topic_num corresponding to a 
    given topic number and
    returns the list of top n occuring concepts
    from the top_concept field
    """
    top_concepts = dfpapers[dfpapers['cluster'] == topic_num]['top_concepts'].tolist()
    flat_concepts = [item for sublist in top_concepts for item in sublist]
    concepts_dict = {c:flat_concepts.count(c) for c in flat_concepts}
    sorted_concepts = sorted(concepts_dict.items(), key=lambda x:x[1], reverse=True)
    return [c[0] for c in sorted_concepts][:n]

def get_yake_cluster_phrases(topic_num:int, n:int=20):
    """
    takes in an integer n corresponding
    to a given topic number and
    returns the list of keyphrases (TopicRank method)
    """
    documents = dfpapers[dfpapers['cluster'] == topic_num]['content'].tolist()
    topic_input = ". ".join(documents)
    #extractor = pke.unsupervised.TextRank()
    kw_extractor = yake.KeywordExtractor(top=n, stopwords=None)
    keywords = kw_extractor.extract_keywords(topic_input)
    #extractor.load_document(input=topic_input,
    #                    language='en',
    #                    normalization=None)

    #extractor.candidate_selection()

    #window = 2
    #use_stems = False
    #extractor.candidate_weighting(window=window,
    #                          use_stems=use_stems)
    #extractor.candidate_weighting()
    #threshold = 0.8
   # keyphrases = extractor.get_n_best(n=20, threshold=threshold)
    #keyphrases = extractor.get_n_best(n=n)
    return [p[0] for p in keywords]

wikiconcepts = df_reduced_topics['topic'].apply(get_cluster_concepts)

wikikeywords = df_reduced_topics['topic'].apply(get_yake_cluster_phrases)

dfpapers['id'] = dfpapers.index
dfinfo = dfpapers[['x','y','id','title','doi','cluster','grants',
                   'locations',
                 'publication_date','keywords','top_concepts']].copy()

centroids = dfinfo.groupby('cluster')[['x','y']].mean().copy()
centroids['concepts'] = wikiconcepts
centroids['cluster'] = centroids.index
centroids['keywords'] = wikikeywords

In [41]:
def wrap_it(x):
    return "<br>".join(textwrap.wrap(x, width=40))
   # return "<br>".join(textwrap.wrap(x.replace(r'\s+', ' '), width=40))

In [42]:
centroids['wrapped_keywords'] = centroids['keywords'].apply(str).apply(wrap_it)
centroids['wrapped_concepts'] = centroids['concepts'].apply(str).apply(wrap_it)

In [43]:
centroids.to_pickle('updatejammingcentroids2d.pkl')

In [44]:
dftriple.to_pickle('updatejammingdftriple2d.pkl')

In [170]:
def get_affils_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by id, ror sorted
    by the some of probablity descending
    """
    # https://learning.oreilly.com/library/view/streamlit-for-data/9781803248226/text/ch004.xhtml
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg.groupby(['id','display_name','country_code',
                     'type'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    dv.reset_index(inplace=True) # map the display_name column with the geo_dict to get lattitude, longitude
    dv['latitude'] = dv['display_name'].apply(lambda x: affil_geo_dict.get(x, (None, None))[0])
    dv['longitude'] = dv['display_name'].apply(lambda x: affil_geo_dict.get(x, (None, None))[1])
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [46]:
dv84, kw84 = get_affils_cluster_sort(dftriple, 1)
print(kw84)
dv84.head(10)

1
['robotic manipulator system', 'Robotic Manipulator', 'Robotic Manipulator Control', 'Robot manipulator', 'Sliding Mode Control', 'Flexible Robotic Manipulator', 'robotic manipulator based', 'redundant robot manipulators', 'manipulator control system', 'Timoshenko robotic manipulator', 'industrial robot manipulator', 'Robot Manipulators Based', 'Robotic Arm Manipulator', 'control robotic manipulators', 'Robot Manipulators Control', 'DOF Robotic Manipulator', 'Soft Robotic Manipulators', 'robot manipulator systems', 'Manipulator Robotic Control', 'Manipulator']


,,,,paper_cluster_score
id,display_name,country_code,type,
https://openalex.org/I204983213,Harbin Institute of Technology,CN,education,83.0
https://openalex.org/I125839683,Beijing Institute of Technology,CN,education,67.0
https://openalex.org/I19820366,Chinese Academy of Sciences,CN,government,51.0
https://openalex.org/I90610280,South China University of Technology,CN,education,50.961082
https://openalex.org/I92403157,University of Science and Technology Beijing,CN,education,49.0
https://openalex.org/I157773358,Sun Yat-sen University,CN,education,47.0
https://openalex.org/I62916508,Technical University of Munich,DE,education,42.0
https://openalex.org/I40542001,University of Ulsan,KR,education,40.0
https://openalex.org/I47508984,Imperial College London,GB,education,40.0


In [171]:
dv84, kw84 = get_affils_cluster_sort(dftriple, 0)
print(kw84)
dv84.head(10)

0
['Aerial Manipulator', 'Aerial Manipulator Robot', 'Aerial Robotic Manipulators', 'Aerial', 'Manipulator', 'Aerial Manipulator System', 'unmanned aerial vehicle', 'aerial manipulator control', 'Control Aerial Manipulator', 'aerial manipulation', 'Aerial Robotic', 'aerial manipulator vehicle', 'aerial vehicle', 'Protocentric Aerial Manipulators', 'proposed aerial manipulator', 'Unmanned aerial', 'Robotic Manipulator', 'control', 'Aerial Robotic System', 'robotic arm']


,id,display_name,country_code,type,paper_cluster_score,latitude,longitude
0,https://openalex.org/I4210098188,Laboratori Guglielmo Marconi (Italy),IT,company,8.0,44.493810,11.338750
1,https://openalex.org/I2801039369,Marconi University,IT,education,8.0,41.912777,12.393333
2,https://openalex.org/I9360294,University of Bologna,IT,education,8.0,44.493810,11.338750
3,https://openalex.org/I118946981,Escuela Politécnica del Ejército,EC,education,7.253707,-0.334050,-78.452170
4,https://openalex.org/I82880672,Beihang University,CN,education,5.609914,39.907500,116.397230
5,https://openalex.org/I79238269,Universidad de Sevilla,ES,education,5.505059,37.382830,-5.973170
6,https://openalex.org/I190497903,Laboratory for Analysis and Architecture of Sy...,FR,facility,5.298005,43.604260,1.443670
7,https://openalex.org/I17866349,Université de Toulouse,FR,education,4.527324,43.604260,1.443670
8,https://openalex.org/I1294671590,French National Centre for Scientific Research,FR,government,4.527324,48.853410,2.348800
9,https://openalex.org/I195740183,University of Moratuwa,LK,education,4.0,6.773000,79.881600


In [48]:
dv84, kw84 = get_affils_cluster_sort(dftriple, 0)
print(kw84)
dv84.head(20)

0
['Aerial Manipulator', 'Aerial Manipulator Robot', 'Aerial Robotic Manipulators', 'Aerial', 'Manipulator', 'Aerial Manipulator System', 'unmanned aerial vehicle', 'aerial manipulator control', 'Control Aerial Manipulator', 'aerial manipulation', 'Aerial Robotic', 'aerial manipulator vehicle', 'aerial vehicle', 'Protocentric Aerial Manipulators', 'proposed aerial manipulator', 'Unmanned aerial', 'Robotic Manipulator', 'control', 'Aerial Robotic System', 'robotic arm']


,,,,paper_cluster_score
id,display_name,country_code,type,
https://openalex.org/I4210098188,Laboratori Guglielmo Marconi (Italy),IT,company,8.0
https://openalex.org/I2801039369,Marconi University,IT,education,8.0
https://openalex.org/I9360294,University of Bologna,IT,education,8.0
https://openalex.org/I118946981,Escuela Politécnica del Ejército,EC,education,7.253707
https://openalex.org/I82880672,Beihang University,CN,education,5.609914
https://openalex.org/I79238269,Universidad de Sevilla,ES,education,5.505059
https://openalex.org/I190497903,Laboratory for Analysis and Architecture of Systems,FR,facility,5.298005
https://openalex.org/I17866349,Université de Toulouse,FR,education,4.527324
https://openalex.org/I1294671590,French National Centre for Scientific Research,FR,government,4.527324


In [49]:
dfinfo = dfpapers[['x','y','id','title','doi','cluster','probability',
                 'publication_date','grants','locations',
                   'keywords','top_concepts']].copy()

In [50]:
dfpapers['primary_location'].iloc[58]

{'is_oa': False,
 'landing_page_url': 'https://doi.org/10.1016/j.robot.2017.05.015',
 'pdf_url': None,
 'source': {'id': 'https://openalex.org/S133768115',
  'display_name': 'Robotics and autonomous systems',
  'issn_l': '0921-8890',
  'issn': ['0921-8890', '1872-793X'],
  'is_oa': False,
  'is_in_doaj': False,
  'host_organization': 'https://openalex.org/P4310320990',
  'host_organization_name': 'Elsevier BV',
  'host_organization_lineage': ['https://openalex.org/P4310320990'],
  'host_organization_lineage_names': ['Elsevier BV'],
  'type': 'journal'},
 'license': None,
 'license_id': None,
 'version': None,
 'is_accepted': False,
 'is_published': False}

In [51]:
dfpapers['locations'].iloc[58]

[{'is_oa': False,
  'landing_page_url': 'https://doi.org/10.1016/j.robot.2017.05.015',
  'pdf_url': None,
  'source': {'id': 'https://openalex.org/S133768115',
   'display_name': 'Robotics and autonomous systems',
   'issn_l': '0921-8890',
   'issn': ['0921-8890', '1872-793X'],
   'is_oa': False,
   'is_in_doaj': False,
   'host_organization': 'https://openalex.org/P4310320990',
   'host_organization_name': 'Elsevier BV',
   'host_organization_lineage': ['https://openalex.org/P4310320990'],
   'host_organization_lineage_names': ['Elsevier BV'],
   'type': 'journal'},
  'license': None,
  'license_id': None,
  'version': None,
  'is_accepted': False,
  'is_published': False}]

In [52]:
dftriple.columns

Index(['id', 'display_name', 'ror', 'country_code', 'type', 'lineage',
       'paper_id', 'paper_raw_affiliation_strings', 'paper_author_position',
       'paper_doi', 'paper_title', 'paper_abstract', 'paper_publication_date',
       'paper_publication_year', 'paper_grants', 'paper_locations',
       'paper_is_corrresponding', 'paper_x', 'paper_y', 'paper_cluster',
       'paper_cluster_score', 'paper_author_id', 'paper_author_display_name',
       'paper_author_orcid'],
      dtype='object')

In [53]:
dftriple[['paper_id','paper_raw_affiliation_strings']].head()

,paper_id,paper_raw_affiliation_strings
0,https://openalex.org/W2740675802,"[Ministry of Education, Key Laboratory of Know..."
1,https://openalex.org/W2740675802,"[Ministry of Education, Key Laboratory of Know..."
2,https://openalex.org/W2740675802,[Department of Electrical and Computer Enginee...
3,https://openalex.org/W2418767125,"[School of Automation, Southeast University, N..."
4,https://openalex.org/W2418767125,[School of Automation and Electrical Engineeri...


In [54]:
dftriple['paper_raw_affiliation_strings'].value_counts(dropna=False)

[School of Information and Communication Engineering, University of Electronic Science and Technology of China, Chengdu, China]                     102
[Sandia National Laboratories, P.O. Box 5800, Albuquerque, New Mexico 87185, USA]                                                                    91
[Sandia National Laboratories, Albuquerque, New Mexico 87185, USA]                                                                                   85
[State Key Laboratory of Electronic Thin Films and Integrated Devices, University of Electronic Science and Technology of China, Chengdu, China]     84
[School of Electrical and Electronic Engineering, Shandong University of Technology, Zibo, China]                                                    80
                                                                                                                                                   ... 
[Pohang University of Science and Technology, Pohang, Gyeongbuk, Korea]                 

In [55]:
dftriple[['paper_id','paper_raw_affiliation_strings']].head()

,paper_id,paper_raw_affiliation_strings
0,https://openalex.org/W2740675802,"[Ministry of Education, Key Laboratory of Know..."
1,https://openalex.org/W2740675802,"[Ministry of Education, Key Laboratory of Know..."
2,https://openalex.org/W2740675802,[Department of Electrical and Computer Enginee...
3,https://openalex.org/W2418767125,"[School of Automation, Southeast University, N..."
4,https://openalex.org/W2418767125,[School of Automation and Electrical Engineeri...


In [56]:
# Group by 'paper_id' and concatenate 'paper_raw_affiliation_strings'
grouped = dftriple.groupby('paper_id')['paper_raw_affiliation_strings'].apply(lambda x: list(set([item for sublist in x for item in sublist]))).reset_index()

# Convert the series back to a dictionary
pap_affils_dict = grouped.set_index('paper_id')['paper_raw_affiliation_strings'].to_dict()

In [57]:
#paper_affils_dict

In [58]:
#pap_affils_dict = dftriple.groupby('paper_id')['paper_raw_affiliation_strings'].\
#apply(lambda x: ' | '.join(x.tolist()))

#pap_affils_dict = dftriple.set_index('paper_id')['paper_raw_affiliation_strings'].to_dict()
import itertools

#pap_affils_dict = dftriple.set_index('paper_id')['paper_raw_affiliation_strings']



      

In [59]:
type(pap_affils_dict)

dict

In [60]:
#pap_affils_dict.head()

In [61]:
pap_authors_dict = dftriple.groupby('paper_id')['paper_author_display_name'].apply(lambda x: x.values)

In [62]:
pap_authors_dict

paper_id
https://openalex.org/W2166534330                   [Hussain Sultan, Eric M. Schwartz]
https://openalex.org/W2252881006                         [Xiaohan Chen, Xiaohan Chen]
https://openalex.org/W2280830725    [Alireza Rahrooh, Scott Shepard, Walter Buchan...
https://openalex.org/W2340996099                                      [John Dennison]
https://openalex.org/W2343923805    [Zhengxin Hou, Lei Liu, Yongji Wang, Jian Huan...
                                                          ...                        
https://openalex.org/W4396816771                                  [Menaouer Bennaoum]
https://openalex.org/W4396877673    [Qinzhe Lv, Hongqin Fan, Yinghai Zhao, Mengdao...
https://openalex.org/W4396877719    [Zhizhen Liu, Xinjie Yu, Zhen Li, Hao Sun, Bei...
https://openalex.org/W4396878689    [Lyubov Gorbacheva, Maxim Zakharov, Andrey Kou...
https://openalex.org/W584406582                              [M. O. Tokhi, Abul Azad]
Name: paper_author_display_name, Length: 7702

In [63]:
type(pap_authors_dict)

pandas.core.series.Series

In [64]:
#dfinfo.head()

In [65]:
dfinfo['affil_list'] = dfinfo['id'].map(pap_affils_dict)

In [66]:
dfinfo['affil_list'].head().iloc[4]

['School of Information Science and Engineering, Qufu Normal University, Rizhao, China',
 'School of Information Science and Engineering, Lanzhou University, Lanzhou, China',
 'Department of Computing, The Hong Kong Polytechnic University, Hong Kong']

In [67]:
dfinfo[['id','affil_list']].iloc[4]

id                             https://openalex.org/W2792852625
affil_list    [School of Information Science and Engineering...
Name: https://openalex.org/W2792852625, dtype: object

In [70]:
dfinfo['author_list'] = dfinfo['id'].map(pap_authors_dict)

In [71]:
dfinfo['wrapped_affil_list'] = dfinfo['affil_list'].apply(str).apply(wrap_it)
dfinfo['wrapped_author_list'] = dfinfo['author_list'].apply(str).apply(wrap_it)

In [72]:
dfinfo['wrapped_keywords'] = dfinfo['keywords'].apply(str).apply(wrap_it)

In [73]:
dfinfo['locations'].iloc[69]

[{'is_oa': False,
  'landing_page_url': 'https://doi.org/10.1016/j.isatra.2020.06.017',
  'pdf_url': None,
  'source': {'id': 'https://openalex.org/S89543202',
   'display_name': 'ISA transactions',
   'issn_l': '0019-0578',
   'issn': ['0019-0578', '1879-2022'],
   'is_oa': False,
   'is_in_doaj': False,
   'host_organization': 'https://openalex.org/P4310320990',
   'host_organization_name': 'Elsevier BV',
   'host_organization_lineage': ['https://openalex.org/P4310320990'],
   'host_organization_lineage_names': ['Elsevier BV'],
   'type': 'journal'},
  'license': None,
  'license_id': None,
  'version': None,
  'is_accepted': False,
  'is_published': False},
 {'is_oa': False,
  'landing_page_url': 'https://pubmed.ncbi.nlm.nih.gov/32595010',
  'pdf_url': None,
  'source': {'id': 'https://openalex.org/S4306525036',
   'display_name': 'PubMed',
   'issn_l': None,
   'issn': None,
   'is_oa': False,
   'is_in_doaj': False,
   'host_organization': 'https://openalex.org/I1299303238',
   'h

In [74]:
def get_source_name(loc_list):
    """
    grab the first item in the list;
    retturn the display name
    """
    try:
        primary = loc_list[0]
        return primary["source"]["display_name"]
    except:
        return None

def get_source_type(loc_list):
    """
    grab the first item in the list;
    return the source type
    """
    try:
        primary = loc_list[0]
        return primary["source"]["type"]
    except:
        return None

In [75]:
dfinfo["source"] = dfinfo["locations"].apply(get_source_name)
dfinfo["source_type"] = dfinfo["locations"].apply(get_source_type)

In [76]:
dfinfo["source"].value_counts()


IEEE transactions on plasma science                                                  303
arXiv (Cornell University)                                                           206
Lecture notes in electrical engineering                                              178
Journal of physics. Conference series                                                175
IEEE access                                                                          135
                                                                                    ... 
2022 7th International Conference on Integrated Circuits and Microsystems (ICICM)      1
Pisʹma v Žurnal tehničeskoj fiziki                                                     1
Transportation research procedia                                                       1
Hyōmen kagaku/Hyoumen kagaku                                                           1
Šumadijski anali                                                                       1
Name: source, Length:

In [77]:
dfinfo["source_type"].value_counts()

journal           4984
conference         746
book series        482
repository         273
ebook platform     151
Name: source_type, dtype: int64

In [78]:
dfinfo[dfinfo["source_type"] == "conference"]

,x,y,id,title,doi,cluster,probability,publication_date,grants,locations,keywords,top_concepts,affil_list,authors_list,wrapped_affil_list,author_list,wrapped_author_list,wrapped_keywords,source,source_type
id,,,,,,,,,,,,,,,,,,,,
https://openalex.org/W4285102237,15.100839,-2.591275,https://openalex.org/W4285102237,Provably Safe Deep Reinforcement Learning for ...,https://doi.org/10.1109/icra46639.2022.9811698,1,1.0,2022-05-23,[],"[{'is_oa': False, 'landing_page_url': 'https:/...","[Safe Deep Reinforcement, Provably Safe Deep, ...","[Reinforcement learning, Reachability, Compute...","[Technical Univer-sity of Munich,Department of...","[Jakob Thumm, Matthias Althoff]","['Technical Univer-sity of<br>Munich,Departmen...","[Jakob Thumm, Matthias Althoff]",['Jakob Thumm' 'Matthias Althoff'],"['Safe Deep Reinforcement', 'Provably<br>Safe ...",2022 International Conference on Robotics and ...,conference
https://openalex.org/W3017045808,13.942735,-4.661314,https://openalex.org/W3017045808,A Systematic Review and Meta-analysis of Robot...,https://doi.org/10.1088/1757-899x/782/4/042055,1,1.0,2020-03-01,[],"[{'is_oa': True, 'landing_page_url': 'https://...","[Systematic Review, Review and Meta-analysis, ...","[Grippers, GRASP, Robotics, Artificial intelli...","[School of Engineering, Southwest Petroleum Un...","[Zhang Long, Jiang Qian, Shuai Tao, Feijuan We...","['School of Engineering, Southwest<br>Petroleu...","[Zhang Long, Jiang Qian, Shuai Tao, Feijuan We...",['Zhang Long' 'Jiang Qian' 'Shuai Tao'<br>'Fei...,"['Systematic Review', 'Review and Meta-<br>ana...",IOP conference series. Materials science and e...,conference
https://openalex.org/W2761006485,15.275157,-3.679164,https://openalex.org/W2761006485,Kinematic singularity avoidance for robot mani...,https://doi.org/10.1109/ccta.2017.8062454,1,1.0,2017-08-01,[],"[{'is_oa': False, 'landing_page_url': 'https:/...","[Kinematic singularity avoidance, singularity ...","[Kinematics, Singularity, Control theory (soci...","[Department of Engineering Cybernetics, Norweg...","[J. Sverdrup-Thygeson, Signe Moe, Kristin Y. P...","['Department of Engineering Cybernetics,<br>No...","[J. Sverdrup-Thygeson, Signe Moe, Kristin Y. P...",['J. Sverdrup-Thygeson' 'Signe Moe'<br>'Kristi...,"['Kinematic singularity avoidance',<br>'singul...",2017 IEEE Conference on Control Technology and...,conference
https://openalex.org/W4285102150,14.262562,-4.263948,https://openalex.org/W4285102150,An Integrated Design Pipeline for Tactile Sens...,https://doi.org/10.1109/icra46639.2022.9812335,1,1.0,2022-05-23,[],"[{'is_oa': False, 'landing_page_url': 'https:/...","[Integrated Design Pipeline, Tactile Sensing R...","[Modular design, Pipeline (software)]",[Computer Science and Artificial Intelligence ...,"[Lara Zlokapa, Yiyue Luo, Jie Xu, Michael Fosh...",['Computer Science and Artificial<br>Intellige...,"[Lara Zlokapa, Yiyue Luo, Jie Xu, Michael Fosh...",['Lara Zlokapa' 'Yiyue Luo' 'Jie Xu'<br>'Micha...,"['Integrated Design Pipeline', 'Tactile<br>Sen...",2022 International Conference on Robotics and ...,conference
https://openalex.org/W2927011308,14.625490,-4.040068,https://openalex.org/W2927011308,URDF Generator for Manipulator Robot,https://doi.org/10.1109/irc.2019.00101,1,1.0,2019-02-01,[],"[{'is_oa': False, 'landing_page_url': 'https:/...","[URDF Generator, robots market grows, Generato...","[Parallel manipulator, Robot, Manipulator (dev...","[Department of Eletronics, Kyung Hee Universit...","[Yeon June Kang, Donghan Kim]","['Department of Eletronics, Kyung Hee<br>Unive...","[Yeon June Kang, Donghan Kim]",['Yeon June Kang' 'Donghan Kim'],"['URDF Generator', 'robots market<br>grows', '...",2019 Third IEEE International Conference on Ro...,conference
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://openalex.org/W3023203661,8.546629,8.577993,https://openalex.org/W3023203661,Simulation Research on New Model of Air-to-Air...,https://doi.org/10.1109/itnec48623.2020.9084793,11,1.0

In [79]:
dfinfo.columns

Index(['x', 'y', 'id', 'title', 'doi', 'cluster', 'probability',
       'publication_date', 'grants', 'locations', 'keywords', 'top_concepts',
       'affil_list', 'authors_list', 'wrapped_affil_list', 'author_list',
       'wrapped_author_list', 'wrapped_keywords', 'source', 'source_type'],
      dtype='object')

In [80]:
dftriple.columns

Index(['id', 'display_name', 'ror', 'country_code', 'type', 'lineage',
       'paper_id', 'paper_raw_affiliation_strings', 'paper_author_position',
       'paper_doi', 'paper_title', 'paper_abstract', 'paper_publication_date',
       'paper_publication_year', 'paper_grants', 'paper_locations',
       'paper_is_corrresponding', 'paper_x', 'paper_y', 'paper_cluster',
       'paper_cluster_score', 'paper_author_id', 'paper_author_display_name',
       'paper_author_orcid'],
      dtype='object')

In [81]:
dfinfo.to_pickle('updatejammingdfinfo2d.pkl')

In [82]:
#dftriple['paper_grants'].value_counts()

In [83]:
def get_funder_names(funder_list):
    """
    funder_list is a list of dictionaries
    with three keys; return the list of 
    unique **funder_display_name**
    values
    """
    try:
        funder_names = list(set([f['funder_display_name'] for f in funder_list]))
        return funder_names    
    except:
        return []
        

In [84]:
dftriple["source"] = dftriple["paper_locations"].apply(get_source_name)
dftriple["source_type"] = dftriple["paper_locations"].apply(get_source_type)
dftriple["funder_list"] = dftriple["paper_grants"].apply(get_funder_names)

In [85]:
dftriple.to_pickle('updatejammingdftriple2d.pkl')

In [86]:
dftriple['source'].value_counts()

IEEE transactions on plasma science                                         1701
Review of scientific instruments online/Review of scientific instruments    1106
Journal of physics. Conference series                                        589
Lecture notes in electrical engineering                                      547
IEEE access                                                                  540
                                                                            ... 
Europhysics letters                                                            1
Fizika tverdogo tela                                                           1
Defense and security analysis                                                  1
Acta crystallographica. Section A, Foundations and advances                    1
Bioprocess engineering                                                         1
Name: source, Length: 1336, dtype: int64

In [87]:
dftriple['source_type'].value_counts()

journal           21220
conference         2811
book series        1540
repository          290
ebook platform      167
Name: source_type, dtype: int64

In [88]:
def get_journals_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by source (where
    source_type == 'journal') sorted
    by the some of probablity descending
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg[dg['source_type'] == 'journal'].groupby(['source'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [89]:
def get_conferences_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by source (where
    source_type == 'journal') sorted
    by the some of probablity descending
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg[dg['source_type'] == 'conference'].groupby(['source'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [90]:
dv84, kw84 = get_journals_cluster_sort(dftriple, 1)
print(kw84)
dv84.head(10)

1
['robotic manipulator system', 'Robotic Manipulator', 'Robotic Manipulator Control', 'Robot manipulator', 'Sliding Mode Control', 'Flexible Robotic Manipulator', 'robotic manipulator based', 'redundant robot manipulators', 'manipulator control system', 'Timoshenko robotic manipulator', 'industrial robot manipulator', 'Robot Manipulators Based', 'Robotic Arm Manipulator', 'control robotic manipulators', 'Robot Manipulators Control', 'DOF Robotic Manipulator', 'Soft Robotic Manipulators', 'robot manipulator systems', 'Manipulator Robotic Control', 'Manipulator']


,paper_cluster_score
source,
IFAC-PapersOnLine,119.0
"Robotikusu, Mekatoronikusu Koenkai koen gaiyoshu",89.0
Applied sciences,87.0
IEEE access,86.0
IEEE robotics & automation letters,69.0
Robotica,68.0
International journal of robust and nonlinear control,58.0
Journal of physics. Conference series,46.0
IEEE transactions on industrial informatics,40.0


# Country - Country Collaborations

want to report back though which countries are involved as well. ok.

In [91]:
def get_country_collaborations_sort(dc:pd.DataFrame, cl:int):
    """
    resticts the dataframe dc to cluster value cl
    and returns the results of paper_id s where there is 
    more than one country_code
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    dv = dg.groupby('paper_id')['country_code'].apply(lambda x: len(set(x.values))).to_frame()
    dc = dg.groupby('paper_id')['country_code'].apply(lambda x: list(set(x.values))).to_frame()
    dc.columns = ['collab_countries']
    dv.columns = ['country_count']
    dv['collab_countries'] = dc['collab_countries']
    dv.sort_values('country_count',ascending=False, inplace=True)
    di = dfinfo.loc[dv.index].copy()
    di['country_count'] = dv['country_count']
    di['collab_countries'] = dv['collab_countries']
    return di[di['country_count'] > 1]

In [92]:
dv = get_country_collaborations_sort(dftriple, 0)
dv

,x,y,id,title,doi,cluster,probability,publication_date,grants,locations,...,affil_list,authors_list,wrapped_affil_list,author_list,wrapped_author_list,wrapped_keywords,source,source_type,country_count,collab_countries
paper_id,,,,,,,,,,,,,,,,,,,,,
https://openalex.org/W3174123873,12.870756,-0.286967,https://openalex.org/W3174123873,"Past, Present, and Future of Aerial Robotic Ma...",https://doi.org/10.1109/tro.2021.3084395,0,1.000000,2022-02-01,[{'funder': 'https://openalex.org/F4320320300'...,"[{'is_oa': True, 'landing_page_url': 'https://...",...,"[Department of Mechanical Engineering, IAMD an...","[Anı́bal Ollero, Marco Tognon, Alejandro Suáre...","['Department of Mechanical Engineering,<br>IAM...","[Anı́bal Ollero, Marco Tognon, Alejandro Suáre...",['Anı́bal Ollero' 'Marco Tognon'<br>'Alejandro...,"['Aerial Robotic Manipulators', 'Robotic<br>Ma...",IEEE transactions on robotics,journal,5,"[NL, KR, FR, CH, ES]"
https://openalex.org/W3013279486,12.830274,-0.375535,https://openalex.org/W3013279486,Design and Development of a Low-Cost Indigenou...,https://doi.org/10.1109/isai-nlp48611.2019.904...,0,0.127045,2019-10-01,[],"[{'is_oa': False, 'landing_page_url': 'https:/...",...,"[Muban Chombueng Rajabhat University,Faculty o...","[Adil Farooq, Muhammad Irfan, Abdullah Saeed, ...","['Muban Chombueng Rajabhat<br>University,Facul...","[Adil Farooq, Muhammad Irfan, Abdullah Saeed, ...",['Adil Farooq' 'Muhammad Irfan'<br>'Abdullah S...,"['Unmanned Ground Vehicle', 'Indigenous<br>Sol...",None,None,4,"[TH, PK, CY, AE]"
https://openalex.org/W4361764889,12.858668,-0.323213,https://openalex.org/W4361764889,Innovative Development of a Flying robot with ...,https://doi.org/10.15439/2022r47,0,0.383097,2022-02-20,[],"[{'is_oa': True, 'landing_page_url': 'https://...",...,"[Institute of Simulation Technology, Le Quy Do...","[Yavor Yotov, Yavor Yotov, Nikolay Zlatov, Nik...","['Institute of Simulation Technology, Le<br>Qu...","[Yavor Yotov, Yavor Yotov, Nikolay Zlatov, Nik...",['Yavor Yotov' 'Yavor Yotov' 'Nikolay<br>Zlato...,"['aerial manipulation platform',<br>'Dexterous...",Annals of Computer Science and Information Sys...,conference,4,"[VN, MY, BG, GB]"
https://openalex.org/W3131230508,12.877010,-0.281814,https://openalex.org/W3131230508,Compliance Control of a Cable-Suspended Aerial...,https://doi.org/10.1109/iros45743.2020.9340703,0,0.793656,2020-10-24,[],"[{'is_oa': False, 'landing_page_url': 'https:/...",...,"[Institute of Robotics and Mechatronics, Germa...","[Chiara Gabellieri, Chiara Gabellieri, Yu. S. ...","['Institute of Robotics and<br>Mechatronics, G...","[Chiara Gabellieri, Chiara Gabellieri, Yu. S. ...",['Chiara Gabellieri' 'Chiara Gabellieri'<br>'Y...,"['Cable-Suspended Aerial Manipulator',<br>'Hie...",None,None,3,"[RU, DE, IT]"
https://openalex.org/W2901687922,12.859335,-0.288682,https://openalex.org/W2901687922,Force control design for a robot manipulator a...,https://doi.org/10.1016/j.ifacol.2018.11.268,0,0.714888,2018-01-01,[],"[{'is_oa': True, 'landing_page_url': 'https://...",...,"[University of Patras, Electrical and Computer...","[Κωνσταντίνος Γκούντας, Dimitris Chaikalis, An...","['University of Patras, Electrical and<br>Comp...","[Κωνσταντίνος Γκούντας, Dimitris Chaikalis, An...",['Κωνσταντίνος Γκούντας' 'Dimitris<br>Chaikali...,"['Unmanned Aerial Vehicle', 'Force<br>control ...",IFAC-PapersOnLine,journal,2,"[GR, AE]"
https://openalex.org/W3033578287,12.865695,-0.310143,https://openalex.org/W3033578287,"Design, modeling, and control of an aerial man...",https://doi.org/10.1002/rob.21963,0,0.609563,2020-06-01,[{'funder': 'https://openalex.org/F4320334627'...,"[{'is_oa': True, 'landing_page_url': 'https://...",...,"[Department of Mechanical Engineering, Univers...","[Salua Hamaza, Ioannis Georgilas, Guillermo He...","['Department of Mechanical Engineering,<br>Uni...","[Salua Hamaza, Ioannis Georgilas, Guillermo He...",['Salua Hamaza' 'Ioannis Georgilas'<br>'Guille...,"['Abstract On‐site inspection',<br>'placement ...",

In [93]:
dfinfo.head()

,x,y,id,title,doi,cluster,probability,publication_date,grants,locations,keywords,top_concepts,affil_list,authors_list,wrapped_affil_list,author_list,wrapped_author_list,wrapped_keywords,source,source_type
id,,,,,,,,,,,,,,,,,,,,
https://openalex.org/W1517236425,16.062922,-3.320433,https://openalex.org/W1517236425,Neural Network Control Of Robot Manipulators A...,https://doi.org/10.1201/9781003062714,1,1.0,2020-08-13,[],"[{'is_oa': False, 'landing_page_url': 'https:/...","[Robot Manipulators, Neural network controller...","[Robot manipulator, Artificial neural network]",NaN,NaN,nan,NaN,nan,"['Robot Manipulators', 'Neural network<br>cont...",CRC Press eBooks,ebook platform
https://openalex.org/W2740675802,16.461128,-3.442048,https://openalex.org/W2740675802,Adaptive Neural Network Control of a Robotic M...,https://doi.org/10.1109/tcyb.2017.2711961,1,1.0,2017-10-01,[{'funder': 'https://openalex.org/F4320320006'...,"[{'is_oa': False, 'landing_page_url': 'https:/...","[Time-Varying Output Constraints, Output Const...","[Control theory (sociology), Lyapunov function...","[Ministry of Education, Key Laboratory of Know...","[Wei He, Haifeng Huang, Shuzhi Sam Ge]","['Ministry of Education, Key Laboratory<br>of ...","[Wei He, Haifeng Huang, Shuzhi Sam Ge]",['Wei He' 'Haifeng Huang' 'Shuzhi Sam<br>Ge'],"['Time-Varying Output Constraints',<br>'Output...",IEEE transactions on cybernetics,journal
https://openalex.org/W2418767125,16.109781,-3.551340,https://openalex.org/W2418767125,Neural Network Control of a Flexible Robotic M...,https://doi.org/10.1109/tsmc.2016.2562506,1,1.0,2017-08-01,[{'funder': 'https://openalex.org/F4320321001'...,"[{'is_oa': False, 'landing_page_url': 'https:/...","[Flexible Robotic Manipulator, Flexible Roboti...","[Control theory (sociology), Deflection (physi...",[School of Automation Engineering and Center f...,"[Changyin Sun, Wei He, Jin‐Woo Hong]",['School of Automation Engineering and<br>Cent...,"[Changyin Sun, Wei He, Jin‐Woo Hong]",['Changyin Sun' 'Wei He' 'Jin‐Woo Hong'],"['Flexible Robotic Manipulator',<br>'Flexible ...","IEEE transactions on systems, man, and cyberne...",journal
https://openalex.org/W2901112449,15.096028,-5.377518,https://openalex.org/W2901112449,Model-Based Reinforcement Learning for Closed-...,https://doi.org/10.1109/tro.2018.2878318,1,1.0,2019-02-01,[],"[{'is_oa': False, 'landing_page_url': 'https:/...","[Soft Robotic Manipulators, Soft Robotic, Mode...","[Control theory (sociology), Kinematics, Reinf...","[BioRobotics Institute, Scuola Superiore Sant’...","[Thomas George Thuruthel, Egidio Falotico, Fed...","['BioRobotics Institute, Scuola<br>Superiore S...","[Thomas George Thuruthel, Egidio Falotico, Fed...",['Thomas George Thuruthel' 'Egidio<br>Falotico...,"['Soft Robotic Manipulators', 'Soft<br>Robotic...",IEEE transactions on robotics,journal
https://openalex.org/W2792852625,15.602239,-3.136606,https://openalex.org/W2792852625,Robot manipulator control using neural network...,https://doi.org/10.1016/j.neucom.2018.01.002,1,1.0,2018-04-01,[{'funder': 'https://openalex.org/F4320321001'...,"[{'is_oa': False, 'landing_page_url': 'https:/...","[neural networks, neural, Robot manipulators, ...","[Artificial neural network, Computer science]",[School of Information Science and Engineering...,"[Long Jin, Shuai Li, Jiguo Yu, Jinbo He]",['School of Information Science and<br>Enginee...,"[Long Jin, Shuai Li, Jiguo Yu, Jinbo He]",['Long Jin' 'Shuai Li' 'Jiguo Yu' 'Jinbo<br>He'],"['neural networks', 'neural', 'Robot<br>manipu...",Neurocomputing,journal


In [94]:
dftriple.head()

,id,display_name,ror,country_code,type,lineage,paper_id,paper_raw_affiliation_strings,paper_author_position,paper_doi,...,paper_x,paper_y,paper_cluster,paper_cluster_score,paper_author_id,paper_author_display_name,paper_author_orcid,source,source_type,funder_list
0,https://openalex.org/I92403157,University of Science and Technology Beijing,https://ror.org/02egmk993,CN,education,[https://openalex.org/I92403157],https://openalex.org/W2740675802,"[Ministry of Education, Key Laboratory of Know...",first,https://doi.org/10.1109/tcyb.2017.2711961,...,16.461128,-3.442048,1,1.0,https://openalex.org/A5022113595,Wei He,https://orcid.org/0000-0002-8944-9861,IEEE transactions on cybernetics,journal,"[Royal Society, National Key Research and Deve..."
1,https://openalex.org/I92403157,University of Science and Technology Beijing,https://ror.org/02egmk993,CN,education,[https://openalex.org/I92403157],https://openalex.org/W2740675802,"[Ministry of Education, Key Laboratory of Know...",middle,https://doi.org/10.1109/tcyb.2017.2711961,...,16.461128,-3.442048,1,1.0,https://openalex.org/A5055142075,Haifeng Huang,https://orcid.org/0000-0002-1615-3779,IEEE transactions on cybernetics,journal,"[Royal Society, National Key Research and Deve..."
2,https://openalex.org/I165932596,National University of Singapore,https://ror.org/01tgyzw49,SG,education,[https://openalex.org/I165932596],https://openalex.org/W2740675802,[Department of Electrical and Computer Enginee...,last,https://doi.org/10.1109/tcyb.2017.2711961,...,16.461128,-3.442048,1,1.0,https://openalex.org/A5069702292,Shuzhi Sam Ge,None,IEEE transactions on cybernetics,journal,"[Royal Society, National Key Research and Deve..."
3,https://openalex.org/I76569877,Southeast University,https://ror.org/04ct4d772,CN,education,[https://openalex.org/I76569877],https://openalex.org/W2418767125,"[School of Automation, Southeast University, N...",first,https://doi.org/10.1109/tsmc.2016.2562506,...,16.109781,-3.55134,1,1.0,https://openalex.org/A5019248683,Changyin Sun,https://orcid.org/0000-0001-9269-334X,"IEEE transactions on systems, man, and cyberne...",journal,[National Natural Science Foundation of China]
4,https://openalex.org/I92403157,University of Science and Technology Beijing,https://ror.org/02egmk993,CN,education,[https://openalex.org/I92403157],https://openalex.org/W2418767125,[School of Automation and Electrical Engineeri...,middle,https://doi.org/10.1109/tsmc.2016.2562506,...,16.109781,-3.55134,1,1.0,https://openalex.org/A5061536058,Wei He,https://orcid.org/0000-0001-8321-8256,"IEEE transactions on systems, man, and cyberne...",journal,[National Natural Science Foundation of China]


In [95]:
jamming_concepts

[{'id': 'https://openalex.org/C2985527887',
  'wikidata': 'https://www.wikidata.org/wiki/Q1587588',
  'display_name': 'Robot manipulator',
  'relevance_score': 16245.7295,
  'level': 3,
  'description': None,
  'works_count': 10153,
  'cited_by_count': 108685,
  'summary_stats': {'2yr_mean_citedness': 1.6,
   'h_index': 130,
   'i10_index': 1527},
  'ids': {'openalex': 'https://openalex.org/C2985527887',
   'wikidata': 'https://www.wikidata.org/wiki/Q1587588',
   'mag': '2985527887',
   'wikipedia': 'https://en.wikipedia.org/wiki/Manipulator%20%28device%29'},
  'image_url': 'https://upload.wikimedia.org/wikipedia/commons/b/b8/GammaGIF.gif',
  'image_thumbnail_url': 'https://upload.wikimedia.org/wikipedia/commons/thumb/b/b8/GammaGIF.gif/100px-GammaGIF.gif',
  'international': {'display_name': {'ar': 'مناور',
    'de': 'Manipulator',
    'en': 'manipulator',
    'et': 'Manipulaator',
    'fa': 'بازوی مکانیکی',
    'fi': 'Manipulaattori',
    'hr': 'Manipulator',
    'it': 'manipolatore',

# Co-authorship Network

Streamlit with pyvis: https://towardsdatascience.com/how-to-deploy-interactive-pyvis-network-graphs-on-streamlit-6c401d4c99db

the data source is dftriple; let a user interactively select which type of graph , the selection of node types, to display. otherwise its just too too much.

Can display works and authors; construct that first:

In [96]:
dftriple.columns

Index(['id', 'display_name', 'ror', 'country_code', 'type', 'lineage',
       'paper_id', 'paper_raw_affiliation_strings', 'paper_author_position',
       'paper_doi', 'paper_title', 'paper_abstract', 'paper_publication_date',
       'paper_publication_year', 'paper_grants', 'paper_locations',
       'paper_is_corrresponding', 'paper_x', 'paper_y', 'paper_cluster',
       'paper_cluster_score', 'paper_author_id', 'paper_author_display_name',
       'paper_author_orcid', 'source', 'source_type', 'funder_list'],
      dtype='object')

group dftriple by paper_id and get a list of all the paper_author_id values. and then from that list get all distinct subsets of two paper_author_ids. accumulate that list. and then we will haave a weighted undirected graph.

file://wsl.localhost/Ubuntu/home/davidd/2023/SWITCHBOARD/switchboard-mitigations-sort/graphvizmaker.html

In [97]:
import networkx as nx
from pyvis.network import Network
import igraph as ig # for getting a layout w/o relying on slow pyvis physics 

In [98]:
dftriple.columns

Index(['id', 'display_name', 'ror', 'country_code', 'type', 'lineage',
       'paper_id', 'paper_raw_affiliation_strings', 'paper_author_position',
       'paper_doi', 'paper_title', 'paper_abstract', 'paper_publication_date',
       'paper_publication_year', 'paper_grants', 'paper_locations',
       'paper_is_corrresponding', 'paper_x', 'paper_y', 'paper_cluster',
       'paper_cluster_score', 'paper_author_id', 'paper_author_display_name',
       'paper_author_orcid', 'source', 'source_type', 'funder_list'],
      dtype='object')

In [99]:
dfinfo.columns

Index(['x', 'y', 'id', 'title', 'doi', 'cluster', 'probability',
       'publication_date', 'grants', 'locations', 'keywords', 'top_concepts',
       'affil_list', 'authors_list', 'wrapped_affil_list', 'author_list',
       'wrapped_author_list', 'wrapped_keywords', 'source', 'source_type'],
      dtype='object')

In [100]:
dfinfo["funder_list"] = dfinfo["grants"].apply(get_funder_names)
dfinfo["wrapped_funder_list"] = dfinfo["funder_list"].apply(str).apply(wrap_it)

In [101]:
dfinfo.to_pickle('updatejammingdfinfo2d.pkl')

In [102]:
dfinfo[['id','keywords','wrapped_keywords','wrapped_funder_list']].head()

,id,keywords,wrapped_keywords,wrapped_funder_list
id,,,,
https://openalex.org/W1517236425,https://openalex.org/W1517236425,"[Robot Manipulators, Neural network controller...","['Robot Manipulators', 'Neural network<br>cont...",[]
https://openalex.org/W2740675802,https://openalex.org/W2740675802,"[Time-Varying Output Constraints, Output Const...","['Time-Varying Output Constraints',<br>'Output...","['Royal Society', 'National Key Research<br>an..."
https://openalex.org/W2418767125,https://openalex.org/W2418767125,"[Flexible Robotic Manipulator, Flexible Roboti...","['Flexible Robotic Manipulator',<br>'Flexible ...",['National Natural Science Foundation of<br>Ch...
https://openalex.org/W2901112449,https://openalex.org/W2901112449,"[Soft Robotic Manipulators, Soft Robotic, Mode...","['Soft Robotic Manipulators', 'Soft<br>Robotic...",[]
https://openalex.org/W2792852625,https://openalex.org/W2792852625,"[neural networks, neural, Robot manipulators, ...","['neural networks', 'neural', 'Robot<br>manipu...","['Hong Kong Polytechnic University',<br>'Natio..."


In [103]:
kw_dict = dfinfo['keywords'].to_dict()

In [104]:
dftriple[['source','source_type']].head()

,source,source_type
0,IEEE transactions on cybernetics,journal
1,IEEE transactions on cybernetics,journal
2,IEEE transactions on cybernetics,journal
3,"IEEE transactions on systems, man, and cyberne...",journal
4,"IEEE transactions on systems, man, and cyberne...",journal


In [105]:
dftriple.head()

,id,display_name,ror,country_code,type,lineage,paper_id,paper_raw_affiliation_strings,paper_author_position,paper_doi,...,paper_x,paper_y,paper_cluster,paper_cluster_score,paper_author_id,paper_author_display_name,paper_author_orcid,source,source_type,funder_list
0,https://openalex.org/I92403157,University of Science and Technology Beijing,https://ror.org/02egmk993,CN,education,[https://openalex.org/I92403157],https://openalex.org/W2740675802,"[Ministry of Education, Key Laboratory of Know...",first,https://doi.org/10.1109/tcyb.2017.2711961,...,16.461128,-3.442048,1,1.0,https://openalex.org/A5022113595,Wei He,https://orcid.org/0000-0002-8944-9861,IEEE transactions on cybernetics,journal,"[Royal Society, National Key Research and Deve..."
1,https://openalex.org/I92403157,University of Science and Technology Beijing,https://ror.org/02egmk993,CN,education,[https://openalex.org/I92403157],https://openalex.org/W2740675802,"[Ministry of Education, Key Laboratory of Know...",middle,https://doi.org/10.1109/tcyb.2017.2711961,...,16.461128,-3.442048,1,1.0,https://openalex.org/A5055142075,Haifeng Huang,https://orcid.org/0000-0002-1615-3779,IEEE transactions on cybernetics,journal,"[Royal Society, National Key Research and Deve..."
2,https://openalex.org/I165932596,National University of Singapore,https://ror.org/01tgyzw49,SG,education,[https://openalex.org/I165932596],https://openalex.org/W2740675802,[Department of Electrical and Computer Enginee...,last,https://doi.org/10.1109/tcyb.2017.2711961,...,16.461128,-3.442048,1,1.0,https://openalex.org/A5069702292,Shuzhi Sam Ge,None,IEEE transactions on cybernetics,journal,"[Royal Society, National Key Research and Deve..."
3,https://openalex.org/I76569877,Southeast University,https://ror.org/04ct4d772,CN,education,[https://openalex.org/I76569877],https://openalex.org/W2418767125,"[School of Automation, Southeast University, N...",first,https://doi.org/10.1109/tsmc.2016.2562506,...,16.109781,-3.55134,1,1.0,https://openalex.org/A5019248683,Changyin Sun,https://orcid.org/0000-0001-9269-334X,"IEEE transactions on systems, man, and cyberne...",journal,[National Natural Science Foundation of China]
4,https://openalex.org/I92403157,University of Science and Technology Beijing,https://ror.org/02egmk993,CN,education,[https://openalex.org/I92403157],https://openalex.org/W2418767125,[School of Automation and Electrical Engineeri...,middle,https://doi.org/10.1109/tsmc.2016.2562506,...,16.109781,-3.55134,1,1.0,https://openalex.org/A5061536058,Wei He,https://orcid.org/0000-0001-8321-8256,"IEEE transactions on systems, man, and cyberne...",journal,[National Natural Science Foundation of China]


In [106]:
dc = dftriple[dftriple['paper_cluster'] == 10].copy()
dc.shape

(153, 27)

In [107]:
[x for row in dc['funder_list'].tolist() for x in row]

['National Natural Science Foundation of China',
 'National Natural Science Foundation of China',
 'National Natural Science Foundation of China',
 'National Natural Science Foundation of China',
 'National Natural Science Foundation of China',
 'National Natural Science Foundation of China',
 'National Natural Science Foundation of China',
 'Fundamental Research Funds for the Central Universities',
 'National Natural Science Foundation of China',
 'Fundamental Research Funds for the Central Universities',
 'National Natural Science Foundation of China',
 'Fundamental Research Funds for the Central Universities',
 'National Natural Science Foundation of China',
 'Fundamental Research Funds for the Central Universities',
 'National Natural Science Foundation of China',
 'Fundamental Research Funds for the Central Universities',
 'National Natural Science Foundation of China',
 'Fundamental Research Funds for the Central Universities',
 'National Natural Science Foundation of China',
 'F

In [108]:
dftriple['funder_list'].value_counts().head()

[]                                                                                                         22394
[National Natural Science Foundation of China]                                                              3841
[U.S. Department of Energy]                                                                                  359
[National Natural Science Foundation of China, Fundamental Research Funds for the Central Universities]      215
[China Postdoctoral Science Foundation, National Natural Science Foundation of China]                        206
Name: funder_list, dtype: int64

In [109]:
kw_dict = dfinfo['keywords'].to_dict()

# add in the affiliations as nodes as well; that row, author, paper, affil. all three get links. ok.
def create_nx_graph(df: pd.DataFrame, cl:int) -> nx.Graph:
    """
    takes the dataframe df, and creates the undirected graph
    from the source and target columns for each row.
    """
    g = nx.Graph() # dc['paper_cluster'] == cl
    dc = df[df['paper_cluster'] == cl]
    author_counts = dc['paper_author_id'].tolist()
    author_counts_dict = {c:author_counts.count(c) for c in author_counts}
    affiliation_counts = dc['id'].tolist()
    affiliation_counts_dict = {c:affiliation_counts.count(c) for c in affiliation_counts}
    source_counts = dc['source'].tolist()
    source_counts_dict = {c:source_counts.count(c) for c in source_counts}
    funder_counts = [x for row in dc['funder_list'].tolist() for x in row]
    funder_counts_dict = {c:funder_counts.count(c) for c in funder_counts}
    for index, row in df[df['paper_cluster'] == cl].iterrows():
        g.add_node(row['paper_id'], group='work', title=row['paper_title'])
        g.add_node(row['paper_author_id'], title=row['paper_author_display_name'],
                   group='author',value = author_counts_dict[row['paper_author_id']])
        g.add_node(row['id'], group='affiliation',
                   title=row['display_name'] + '\n' + row['country_code'],
                  value = affiliation_counts_dict[row['id']])
        if row['source']:
            g.add_node(row['source'], group=row['source_type'],
                      title=row['source'] + ' :\n ' + row['source_type'],
                      value=source_counts_dict[row['source']])
            g.add_edge(
                row['paper_id'],
                row['source'],
                title=row['paper_title'] + ' :\n ' + str(row['paper_publication_date']) +  \
                ' :\n' + row['source'] + ' :\n ' + \
                row['source_type'],
              #  weight = df[(df['paper_id'] == row['paper_id']) & \
              #              (df['source'] == row['source'])]['paper_cluster_score'].sum()
               # weight = row['paper_cluster_score']
            )
            g.add_edge(
                row['paper_author_id'],
                row['source'],
                title=row['paper_author_display_name'] + ':\n' + row['source'],
             #   weight = df[(df['paper_author_id'] == row['paper_author_id']) & \
              #              (df['source'] == row['source'])]['paper_cluster_score'].sum()
               # weight = row['paper_cluster_score']
            )
        if len(row['funder_list']) > 0:
            for f in row['funder_list']:
                g.add_node(f, group='funder',
                          title=str(f),
                          value = founder_counts_dict[f]),
                g.add_edge(
                       row['paper_id'],
                       f,
                       title=row['paper_title'] + ':\n ' +  str(row['paper_publication_date']) + \
                       ' :\n' + str(f),
                  #  weight = row['paper_cluster_score']
                   )
                g.add_edge(
                       f,
                       row['paper_author_id'],
                       title=row['paper_author_display_name'] + ' :\n ' + \
                       str(f),
                  #  weight = row['paper_cluster_score']
                       
                   )
                g.add_edge(
                       f,
                       row['id'],
                       title=row['display_name'] + '\n' + row['country_code'] + ' :\n ' + \
                       str(f)  ,
                  #  weight = row['paper_cluster_score']
                   )  
                if row["source"]:
                    g.add_edge(
                        f,
                        row["source"],
                        title=row["source"] + ' :\n' + str(f),
                     #   weight = row['paper_cluster_score']
                    )
        g.nodes[row['paper_id']]['title'] = (
            row['paper_title'] + ' :\n ' + str(row['paper_publication_date'] + ':\n' + 
            '\n'.join(kw_dict[row['paper_id']]))
        )
        g.nodes[row['paper_author_id']]['title'] = (
            row['paper_author_display_name']
        )
        g.add_edge(
            row['paper_id'],
            row['paper_author_id'],
        title=row['paper_title'] + ' :\n ' + row['paper_author_display_name'] + ' :\n ' + \
            row['paper_raw_affiliation_string'],
         #   weight = row['paper_cluster_score']
        )
        g.add_edge(
            row['paper_author_id'],
            row['id'],
            title=row['paper_author_display_name'] + ' :\n ' + \
            row['display_name'] + ' :\n ' + row['country_code'],
          #  weight = row['paper_cluster_score']
        )
        g.add_edge(
            row['paper_id'],
            row['id'],
            title=row['paper_title'] + ' :\n ' + str(row['paper_publication_date']) + ':\n' + 
            row['display_name'] + ' :\n ' + row['country_code'],
         #   weight = row['paper_cluster_score']
        )
        
    g_ig = ig.Graph.from_networkx(g) # assign 'x', and 'y' to g before returning
    #layout = g_ig.layout_auto()
    #layout = g_ig.layout_davidson_harel()
    layout = g_ig.layout_umap(min_dist = 2, epochs = 500)
    # https://igraph.org/python/tutorial/0.9.6/visualisation.html
    coords = layout.coords
    allnodes = list(g.nodes())
    coords_dict = {allnodes[i]:(coords[i][0], coords[i][1]) for i in range(len(allnodes))}
    for i in g.nodes():
        g.nodes[i]['x'] = 250 * coords_dict[i][0] # the scale factor needed 
        g.nodes[i]['y'] = 250 * coords_dict[i][1]
    return g
                

In [110]:
def create_pyvis_html(cl: int, filename: str = "pyvis_coauthorships_graph.html"):
    """
    wrapper function that calls create_nx_graph to finally 
    produce an interactive pyvis standalone html file
    """
    g_nx = create_nx_graph(dftriple, cl);
    h = Network(height="1000px",
          #  heading="Mitigations and Techniques Relationships",
                width="100%",
                cdn_resources="remote", # can grab the visjs library to make this local if needed
            # probably should
                bgcolor="#222222",
            neighborhood_highlight=True,
              # default_node_size=1,
                font_color="white",
                directed=False,
               # select_menu=True,
                filter_menu=True,
                notebook=False,
               )
    #h.repulsion()
    h.from_nx(g_nx, show_edge_weights=False)
    #h.barnes_hut()
    #h.repulsion(node_distance=40,
    #            central_gravity=-0.2, spring_length=5, spring_strength=0.005, damping=0.09)
    neighbor_map = h.get_adj_list()
   # for node in h.nodes:
   #     if node['group'] == 'author':
   #         a = list(neighbor_map[node["id"]]) # want to insert a "\n" into every third element of a
   #     if node['group'] == 'work':
   #         a = list(neighbor_map[node["id"]])
   #     i = 3
   #     while i < len(a):
   #         a.insert(i, "\n")
   #         i += 4
   #     node["title"] += "\n Neighbors: \n" + " | ".join(a)
   #     node["value"] = len(neighbor_map[node["id"]]) 
# "physics": {
#    "enabled": false
#  },
    h.set_options(
    """
const options = {
  "interaction": {
    "navigationButtons": false
  },
 "physics": {
     "enabled": false
 },
  "edges": {
    "color": {
        "inherit": true
    },
    "setReferenceSize": null,
    "setReference": {
        "angle": 0.7853981633974483
    },
    "smooth": {
        "forceDirection": "none"
    }
  }
  }
    """
    )
    #h.show_buttons(filter_=['physics'])
  #  h.barnes_hut()
    #h.repulsion()
    try:
        path = './tmp'
        h.save_graph(f"{path}/{filename}")
        HtmlFile = open(f"{path}/{filename}","r",
                        encoding='utf-8')
    except:
        h.save_graph(f"{filename}")
        HtmlFile = open(f"{filename}", "r",
                        encoding="utf-8")
    return h

In [111]:
dfinfo.shape

(9301, 22)

In [112]:
dfinfo.columns

Index(['x', 'y', 'id', 'title', 'doi', 'cluster', 'probability',
       'publication_date', 'grants', 'locations', 'keywords', 'top_concepts',
       'affil_list', 'authors_list', 'wrapped_affil_list', 'author_list',
       'wrapped_author_list', 'wrapped_keywords', 'source', 'source_type',
       'funder_list', 'wrapped_funder_list'],
      dtype='object')

In [113]:
dfinfo[['author_list','authors_list']].head()

,author_list,authors_list
id,,
https://openalex.org/W1517236425,NaN,NaN
https://openalex.org/W2740675802,"[Wei He, Haifeng Huang, Shuzhi Sam Ge]","[Wei He, Haifeng Huang, Shuzhi Sam Ge]"
https://openalex.org/W2418767125,"[Changyin Sun, Wei He, Jin‐Woo Hong]","[Changyin Sun, Wei He, Jin‐Woo Hong]"
https://openalex.org/W2901112449,"[Thomas George Thuruthel, Egidio Falotico, Fed...","[Thomas George Thuruthel, Egidio Falotico, Fed..."
https://openalex.org/W2792852625,"[Long Jin, Shuai Li, Jiguo Yu, Jinbo He]","[Long Jin, Shuai Li, Jiguo Yu, Jinbo He]"


In [114]:
del dfinfo['authors_list']

In [115]:
dfinfo.shape

(9301, 21)

In [116]:
dfinfo[['cluster','probability','publication_date']].head()

,cluster,probability,publication_date
id,,,
https://openalex.org/W1517236425,1,1.0,2020-08-13
https://openalex.org/W2740675802,1,1.0,2017-10-01
https://openalex.org/W2418767125,1,1.0,2017-08-01
https://openalex.org/W2901112449,1,1.0,2019-02-01
https://openalex.org/W2792852625,1,1.0,2018-04-01


In [117]:
dftime = dfinfo[['cluster','probability','publication_date']].copy()

In [118]:
dftime['publication_datetime'] = pd.to_datetime(dftime['publication_date'])

In [119]:
dftime.head()

,cluster,probability,publication_date,publication_datetime
id,,,,
https://openalex.org/W1517236425,1,1.0,2020-08-13,2020-08-13
https://openalex.org/W2740675802,1,1.0,2017-10-01,2017-10-01
https://openalex.org/W2418767125,1,1.0,2017-08-01,2017-08-01
https://openalex.org/W2901112449,1,1.0,2019-02-01,2019-02-01
https://openalex.org/W2792852625,1,1.0,2018-04-01,2018-04-01


In [120]:
def get_time_series(dg, cl:int):
    """
    takes dg and the cluster number cl
    and returns a time series chart
    by month, y-axis is the article count
    """
    dftime = dg[dg.cluster == cl][['cluster','probability','publication_date']].copy()
    dftime['date'] = pd.to_datetime(dftime['publication_date'])
    dftime.sort_values('date', inplace=True)
    #by_month = pd.to_datetime(dftime['date']).dt.to_period('M').value_counts().sort_index()
    #by_month.index = pd.PeriodIndex(by_month.index)
    #df_month = by_month.rename_axis('month').reset_index(name='counts')
    return dftime

In [121]:
dfinfo.cluster.value_counts().head()

 1     1898
-1     1509
 11    1409
 61     396
 39     353
Name: cluster, dtype: int64

In [122]:
df_month_21 = get_time_series(dfinfo, 12)
df_month_21.head()

,cluster,probability,publication_date,date
id,,,,
https://openalex.org/W3185963990,12,0.148192,2017-03-31,2017-03-31
https://openalex.org/W4312408816,12,1.000000,2017-04-10,2017-04-10
https://openalex.org/W2762530402,12,1.000000,2017-07-03,2017-07-03
https://openalex.org/W2786347723,12,1.000000,2018-02-15,2018-02-15
https://openalex.org/W2914873355,12,1.000000,2018-09-02,2018-09-02


In [123]:
import altair as alt
#alt.data_transformers.enable("data_server")

In [124]:
alt.Chart(df_month_21).mark_line().transform_fold(
    ['probability']
).encode(
    x = 'yearmonth(date):T',
    y = 'sum(value):Q',
    color='key:N'
)

alt.Chart(...)

In [125]:
sources_list = dftriple['source'].unique().tolist()
type(sources_list), len(sources_list)

(list, 1337)

In [126]:
Sources().random()

{'id': 'https://openalex.org/S108764191',
 'issn_l': '1759-7137',
 'issn': ['1759-7137', '1759-7145'],
 'display_name': 'Journal of screenwriting',
 'host_organization': 'https://openalex.org/P4310311755',
 'host_organization_name': 'Intellect',
 'host_organization_lineage': ['https://openalex.org/P4310311755'],
 'works_count': 380,
 'cited_by_count': 916,
 'summary_stats': {'2yr_mean_citedness': 0.3088235294117647,
  'h_index': 14,
  'i10_index': 25},
 'is_oa': False,
 'is_in_doaj': False,
 'ids': {'openalex': 'https://openalex.org/S108764191',
  'issn_l': '1759-7137',
  'issn': ['1759-7137', '1759-7145'],
  'mag': '108764191',
  'wikidata': 'https://www.wikidata.org/entity/Q61678276'},
 'homepage_url': 'http://www.intellectbooks.co.uk/journals/view-Journal,id=182',
 'apc_prices': None,
 'apc_usd': None,
 'country_code': 'GB',
 'societies': [],
 'alternate_titles': ['JOSC'],
 'abbreviated_title': None,
 'type': 'journal',
 'topics': [{'id': 'https://openalex.org/T13216',
   'display_n

In [127]:
def get_source_json(s:str):
    """
    s is an openalex Sources display_name
    return that Sources object
    """
    source_json = Sources().search_filter(display_name = s).get()
    a = source_json[0]['type']
    if "homepage_url" in source_json[0] and source_json[0]['homepage_url']:
        print(f"{s} has homepage_url and type {source_json[0]['type']}")
        return source_json[0]["homepage_url"]
    else:
        return None

In [128]:
sources_list[5]

'IEEE transactions on control systems technology'

In [129]:
sj0 = get_source_json(sources_list[5])
sj0

IEEE transactions on control systems technology has homepage_url and type journal


'http://www.ieeecss.org/publications/tcst'

In [130]:
def get_display_page_dict(sl:list):
    """
    sl is a list of Sources display_name values
    returns the dictionary mapping
    display_names with homepage_url values."""
    mapping_dict = dict()
    for s in tqdm(sl):
        try:
            mapping_dict[s] = get_source_json(s)
        except:
            pass
    return mapping_dict

In [131]:
source_page_dict = get_display_page_dict(sources_list)

  0%|                                                                        | 1/1337 [00:01<29:47,  1.34s/it]

IEEE transactions on cybernetics has homepage_url and type journal


  0%|▏                                                                       | 3/1337 [00:02<14:48,  1.50it/s]

IEEE transactions on robotics has homepage_url and type journal


  0%|▏                                                                       | 4/1337 [00:02<13:34,  1.64it/s]

...

Bulletin of "Carol I" National Defense University has homepage_url and type journal


100%|████████████████████████████████████████████████████████████████████▉| 1336/1337 [12:47<00:00,  1.28it/s]

Aviation has homepage_url and type journal


100%|█████████████████████████████████████████████████████████████████████| 1337/1337 [12:48<00:00,  1.74it/s]

Sistemi ozbroênnâ ì vìjsʹkova tehnìka has homepage_url and type journal


In [132]:
len(source_page_dict)

1141

In [133]:
import pickle

with open("updatesource_page_dict.pkl", "wb") as f:
    pickle.dump(source_page_dict, f)

In [134]:
with open("updatesource_page_dict.pkl", "rb") as f:
    source_dict = pickle.load(f)

In [135]:
def get_journals_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by source (where
    source_type == 'journal') sorted
    by the some of probablity descending
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg[dg['source_type'] == 'journal'].groupby(['source'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    dv['journal'] = dv.index
    dv['hompage_url'] = dv['journal'].map(source_dict)
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [136]:
dv, kw = get_journals_cluster_sort(dftriple, 4)
dv.head()

4


,paper_cluster_score,journal,hompage_url
source,,,
JASA express letters,11.524176,JASA express letters,None
The Journal of the Acoustical Society of America/The journal of the Acoustical Society of America,10.562588,The Journal of the Acoustical Society of Ame...,http://asadl.org/jasa
IEEE journal of oceanic engineering,10.407208,IEEE journal of oceanic engineering,http://ieeexplore.ieee.org/servlet/opac?punumb...
Mathematical problems in engineering,9.0,Mathematical problems in engineering,https://www.hindawi.com/journals/mpe/
Remote sensing,6.0,Remote sensing,https://www.journals.elsevier.com/remote-sensi...


In [137]:
def get_conferences_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by source (where
    source_type == 'journal') sorted
    by the some of probablity descending
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg[dg['source_type'] == 'conference'].groupby(['source'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    dv['conference'] = dv.index
    dv['homepage_url'] = dv['conference'].map(source_dict)
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [138]:
dv, kw = get_conferences_cluster_sort(dftriple, 4)
dv.head()

4


,paper_cluster_score,conference,homepage_url
source,,,
"OCEANS 2022, Hampton Roads",7.0,"OCEANS 2022, Hampton Roads",NaN


In [140]:
"IOP conference series" in source_dict

False

In [141]:
len(source_dict)

1141

In [142]:
affils_list = dftriple['display_name'].unique().tolist()
type(affils_list), len(affils_list)

(list, 3074)

In [143]:
affils_list[0]

'University of Science and Technology Beijing'

In [147]:
 affil_json = Institutions().search_filter(display_name = affils_list[0]).get()

In [149]:
#affil_json

In [150]:
affils_list = dftriple['display_name'].unique().tolist()
type(affils_list), len(affils_list)

affil_json = Institutions().search_filter(display_name = affils_list[1]).get()

def get_affil_json(s:str):
    """
    s is an openalex Sources display_name
    return that Sources object
    """
    affil_json = Institutions().search_filter(display_name = s).get()
    #a = source_json[0]['type']
    if "geo" in affil_json[0]:
       # print(f"{s} has homepage_url and type {source_json[0]['type']}")
        return affil_json[0]["geo"]["latitude"], affil_json[0]["geo"]["longitude"]
    else:
        return None, None
    
def get_display_geo_dict(sl:list):
    """
    sl is a list of Institution display_name values
    returns the dictionary mapping
    display_names with (latitude, longitude) values."""
    mapping_dict = dict()
    for s in tqdm(sl):
        try:
            mapping_dict[s] = get_affil_json(s)
        except:
            pass
    return mapping_dict

affil_geo_dict = get_display_geo_dict(affils_list)


import pickle

with open("updateaffil_geo_dict.pkl", "wb") as f:
    pickle.dump(affil_geo_dict, f)

100%|█████████████████████████████████████████████████████████████████████| 3074/3074 [26:29<00:00,  1.93it/s]


In [151]:
len(affil_geo_dict)

2957

In [154]:
dftriple.columns

Index(['id', 'display_name', 'ror', 'country_code', 'type', 'lineage',
       'paper_id', 'paper_raw_affiliation_strings', 'paper_author_position',
       'paper_doi', 'paper_title', 'paper_abstract', 'paper_publication_date',
       'paper_publication_year', 'paper_grants', 'paper_locations',
       'paper_is_corrresponding', 'paper_x', 'paper_y', 'paper_cluster',
       'paper_cluster_score', 'paper_author_id', 'paper_author_display_name',
       'paper_author_orcid', 'source', 'source_type', 'funder_list'],
      dtype='object')

In [155]:
dftriple[['paper_title','paper_abstract']].head()

,paper_title,paper_abstract
0,Adaptive Neural Network Control of a Robotic M...,The control problem of an uncertain n -degrees...
1,Adaptive Neural Network Control of a Robotic M...,The control problem of an uncertain n -degrees...
2,Adaptive Neural Network Control of a Robotic M...,The control problem of an uncertain n -degrees...
3,Neural Network Control of a Flexible Robotic M...,Adaptive neural networks (NNs) are employed fo...
4,Neural Network Control of a Flexible Robotic M...,Adaptive neural networks (NNs) are employed fo...


In [156]:
dfinfo.columns

Index(['x', 'y', 'id', 'title', 'doi', 'cluster', 'probability',
       'publication_date', 'grants', 'locations', 'keywords', 'top_concepts',
       'affil_list', 'wrapped_affil_list', 'author_list',
       'wrapped_author_list', 'wrapped_keywords', 'source', 'source_type',
       'funder_list', 'wrapped_funder_list'],
      dtype='object')

In [157]:
#affil_geo_dict

need to add the abstract to dfinfo

In [158]:
dftriple[['paper_title','paper_abstract']].head()

,paper_title,paper_abstract
0,Adaptive Neural Network Control of a Robotic M...,The control problem of an uncertain n -degrees...
1,Adaptive Neural Network Control of a Robotic M...,The control problem of an uncertain n -degrees...
2,Adaptive Neural Network Control of a Robotic M...,The control problem of an uncertain n -degrees...
3,Neural Network Control of a Flexible Robotic M...,Adaptive neural networks (NNs) are employed fo...
4,Neural Network Control of a Flexible Robotic M...,Adaptive neural networks (NNs) are employed fo...


In [161]:
dfinfo['abstract'] = dfinfo['title'].map(dftriple.set_index('paper_title')['paper_abstract'].to_dict())

In [162]:
dfinfo[['title','abstract']].head(10)

,title,abstract
id,,
https://openalex.org/W1517236425,Neural Network Control Of Robot Manipulators A...,NaN
https://openalex.org/W2740675802,Adaptive Neural Network Control of a Robotic M...,The control problem of an uncertain n -degrees...
https://openalex.org/W2418767125,Neural Network Control of a Flexible Robotic M...,Adaptive neural networks (NNs) are employed fo...
https://openalex.org/W2901112449,Model-Based Reinforcement Learning for Closed-...,Dynamic control of soft robotic manipulators i...
https://openalex.org/W2792852625,Robot manipulator control using neural network...,Robot manipulators are playing increasingly si...
https://openalex.org/W2608224837,Adaptive Integral Sliding Mode Control With Ti...,This paper presents an adaptive robust control...
https://openalex.org/W2559957666,Distributed Task Allocation of Multiple Robots...,The problem of dynamic task allocation in a di...
https://openalex.org/W3099523296,Adaptive Control of Robot Manipulators With Un...,NaN
https://openalex.org/W2783089576,PDE Model-Based Boundary Control Design for a ...,This brief considers the control design proble...


In [163]:
dfinfo.to_pickle('updatejammingdfinfo2d.pkl')

Now try to recreate the pydeck column chart a la DPS-TTU instead of the flat 2d scatter plot. ok.
can also maybe add arcs in between collaborations. maybe. 

In [164]:
dftriple.columns

Index(['id', 'display_name', 'ror', 'country_code', 'type', 'lineage',
       'paper_id', 'paper_raw_affiliation_strings', 'paper_author_position',
       'paper_doi', 'paper_title', 'paper_abstract', 'paper_publication_date',
       'paper_publication_year', 'paper_grants', 'paper_locations',
       'paper_is_corrresponding', 'paper_x', 'paper_y', 'paper_cluster',
       'paper_cluster_score', 'paper_author_id', 'paper_author_display_name',
       'paper_author_orcid', 'source', 'source_type', 'funder_list'],
      dtype='object')

In [168]:
affil_geo_dict

{'University of Science and Technology Beijing': (39.9075, 116.39723),
 'National University of Singapore': (1.28967, 103.85007),
 'Southeast University': (32.06167, 118.77778),
 'University of Electronic Science and Technology of China': (30.66667,
  104.06667),
 "Sant'Anna School of Advanced Studies": (43.721012, 10.402786),
 'Khalifa University of Science and Technology': (24.447926, 54.394863),
 'Lanzhou University': (36.05701, 103.83987),
 'Hong Kong Polytechnic University': (22.27832, 114.17469),
 'Qufu Normal University': (35.59667, 116.99111),
 'Daegu Gyeongbuk Institute of Science and Technology': (35.87028, 128.59111),
 'Korea Institute of Robot and Convergence': (36.022697, 129.32236),
 'Bohai University': (41.087646, 121.1186),
 'Universidad de Sevilla': (37.38283, -5.97317),
 'ETH Zurich': (47.36667, 8.55),
 'Seoul National University': (37.460155, 126.9519),
 'Université de Toulouse': (43.60426, 1.44367),
 'French National Centre for Scientific Research': (48.85341, 2.348

In [172]:
dvaffils, kwwaffils = get_affils_cluster_sort(dftriple, 10)

10


In [173]:
#affil_geo_dict['University of Electronic Science and Technology of China']

In [174]:
dvaffils.head()

,id,display_name,country_code,type,paper_cluster_score,latitude,longitude
0,https://openalex.org/I150229711,University of Electronic Science and Technolog...,CN,education,36.93073,30.66667,104.06667
1,https://openalex.org/I170215575,National University of Defense Technology,CN,education,15.378279,28.19874,112.97087
2,https://openalex.org/I75390827,Beijing University of Chemical Technology,CN,education,8.487354,39.90750,116.39723
3,https://openalex.org/I82880672,Beihang University,CN,education,8.368248,39.90750,116.39723
4,https://openalex.org/I122991210,Institute of Information Science and Technologies,IT,facility,8.0,43.70853,10.40360


ok, can color by type, height and size given by paper_cluster_score

may need to add in the mapbox token. ok.

follow along the treatment here: https://github.com/rs21io/ttu-paths/blob/main/suescoreshypothesis.ipynb

mapbox_token="pk.eyJ1IjoibWFwYm94cnMyMSIsImEiOiJjamdkdTU1MTIwMTM2Mnhxa3Y3ZXZ3eGt3In0.PtflK7MObAbmwY1E__H7Fg"

In [176]:
import pydeck as pdk

view = pdk.data_utils.compute_view(dvaffils[["longitude", "latitude"]])

mapbox_token="pk.eyJ1IjoibWFwYm94cnMyMSIsImEiOiJjamdkdTU1MTIwMTM2Mnhxa3Y3ZXZ3eGt3In0.PtflK7MObAbmwY1E__H7Fg"